In [12]:
import numpy as np
import pandas as pd


genreList = ['Unknown','Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
users = pd.read_csv('./ml-100k/u.user',sep = '|',header = None, index_col=False, names = ['UserID','Age','Gender','Occupation','Zip-code'],engine='python')
movies = pd.read_csv('./ml-100k/u.item',sep = '|',header = None, index_col=False, names = ['MovieID','Title','ReleaseDate','VideoDate','IMDBURL'] + genreList ,engine='python')
ratings = pd.read_csv('./ml-100k/u.data',sep = '\t',header = None, index_col=False,names =['UserID','MovieID','Rating','Timestamp'] ,engine='python')
ratings = ratings.drop('Timestamp',axis=1)
users = users.drop('Zip-code',axis=1)
users = users.drop('Occupation',axis=1)
users = users.replace(to_replace=['M','F'],value=[1,0])


In [13]:
def get_ratings_mov(MovieID):
    answer = ratings[ratings.MovieID == MovieID]
    return answer
def get_ratings_usr(UserID):
    answer = ratings[ratings.UserID == UserID]
    return answer
def get_rating_umv(UserID, MovieID):
    users = get_ratings_usr(UserID)
    answer = users[users.MovieID == MovieID]
    if answer.empty:
        return None
    return answer
def get_movies_gnr(genre):
    answer = movies[getattr(movies,genre) == 1]
    return answer
def get_movies_mov(MovieID):
    answer = movies[movies.MovieID == MovieID]
    return answer
def get_users_age(Age):
    answer = users[users.Age == Age]
    return answer
def get_users_gnd(Gender):
    answer = users[users.Gender == Gender]
    return answer
def get_users_usr(UserID):
    answer = users[users.UserID == UserID]
    return answer

In [14]:
def three_year_mean(user_ID,movie_ID):
    user_age = get_users_usr(user_ID).Age.values[0]
    Xusers = get_users_age(user_age).append(get_users_age(user_age+1)).append(get_users_age(user_age-1))
    Yusers = pd.DataFrame(columns = ['UserID', 'MovieID', 'Rating'])
    for ind,val in Xusers.iterrows():
        Yusers = Yusers.append(ratings.loc[ratings['UserID'] == val[0]])
    Ydata = Yusers[Yusers.MovieID == movie_ID]
    return np.mean(Ydata.Rating)

In [15]:
def gender_mean(user_ID,movie_ID):
    user_gnd = get_users_usr(user_ID).Gender.values[0]
    Xusers = get_users_gnd(user_gnd)
    Yusers = pd.DataFrame(columns = ['UserID', 'MovieID', 'Rating'])
    for ind,val in Xusers.iterrows():
        Yusers = Yusers.append(ratings.loc[ratings['UserID'] == val[0]])
        if ind > 100:
            break
    Ydata = Yusers[Yusers.MovieID == movie_ID]
    return np.mean(Ydata.Rating)

In [16]:
def user_mean(user_ID):
    Ydata = get_ratings_usr(user_ID)
    return np.mean(Ydata.Rating)
def movie_mean(movie_ID):
    Ydata = get_ratings_mov(movie_ID)
    return np.mean(Ydata.Rating)

In [90]:
X = np.zeros((100000,4))
Y = np.zeros((100000,1))
for ind,val in ratings.iterrows():
    if ind >= 100000:
        break
    user_ID = val[0]
    movie_ID = val[1]
    rt = val[2]
    Y[ind] = rt
    X[ind] = [three_year_mean(user_ID,movie_ID), gender_mean(user_ID,movie_ID), user_mean(user_ID),movie_mean(movie_ID)]
X = np.nan_to_num(X)


In [91]:
from sklearn.model_selection import train_test_split
X.reshape(30000,4)
Y.reshape(30000,1)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20)


In [114]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(16,12,8,4),max_iter=2000)
mlp.fit(X_train,Y_train.ravel())


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(16, 12, 8, 4), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [117]:
from sklearn.metrics import mean_squared_error, explained_variance_score

predictions = mlp.predict(X_test)
print(np.sqrt(mean_squared_error(Y_test,predictions)))
print(np.sqrt( explained_variance_score(Y_test,predictions)))


0.85698151172
0.67511105814
